# 策略说明

    开仓条件：
        1. 股价沿5日均线上线
        2. 不存在前一周期rsi超标、当前周期rsi超标以及上涨3%后rsi超标情况。rsi指标处于90%高位为超标
    平仓条件：
        1. rsi 30分钟线高位,或
        2. -5%止损，或
        3. 30分钟两次上攻失败,或
        4. 5天内两次上攻失败

In [2]:
from alpha.notebook import *
from alpha.strategies.z05 import Z05
await init_notebook()

In [3]:
async def scan(dt, n):
    end = tf.day_shift(arrow.get(dt), 0)
    nbars = n + 5 - 1
    start = tf.day_shift(end, -nbars + 1)
    
    results = []
    for code in Securities().choose(["stock"]):
        sec = Security(code)
        try:
            bars = await sec.load_bars(start, end, FrameType.DAY)
            msr = maline_support_ratio(bars["close"], 5, n)
            bd = real_body(bars)
            d1 = ma_d1(bars["close"], 5)[-1]
            d2 = ma_d2(bars["close"], 5)[-1]
            results.append([sec.display_name, code, msr, bd, d1, d2])
        except Exception as e:
            print(e)
            continue
            
    return results
            
results = await scan("2021-10-26", 7)
say("finished")

sh: 1: aplay: not found


In [ ]:
df = pd.DataFrame(results, columns=["name", "code", "msr", "body", "d1", "d2"])

In [ ]:
df.nlargest(20, ["msr", "br", "d2", "d1"])[df.d2>0]

# Z05

## 持仓3天强平

In [ ]:
z05 = Z05(holding_days=3)
summary = await z05.backtest("2021-10-21", "2021-10-21")

In [ ]:
summary

In [ ]:
z05.trades

## 持仓4天强平

In [ ]:
z05 = Z05(holding_days=4)
summary = await z05.backtest("2021-10-21", "2021-10-21")

In [ ]:
summary

In [ ]:
z05.trades

# 昨天的信号

In [ ]:
z05 = Z05(holding_days=3)

end = arrow.get("2021-10-27").date()
await z05.scan(end)

In [ ]:
z05.long_orders

# 30日回测

持仓最长10天

In [ ]:
from itertools import product
import random
import logging

rootLogger = logging.getLogger()
rootLogger.setLevel(logging.WARNING)

rsi = [70, 75, 80, 85]
prsi = [0.7, 0.75, 0.8, 0.85]
rsi3 = [80, 85, 90, 95]
stop_loss = [-0.05, -0.1, -0.15]
msr = [0.6, 0.7, 0.8, 0.9, 1]
bcr = [0.6, 0.7, 0.8, 0.9, 1]
d1 = np.arange(1,4)/100

all_perms = list(product(*(rsi, prsi, rsi3, stop_loss, msr, bcr, d1)))

results = []

for params in random.sample(all_perms, int(0.1 * len(all_perms))):
    rsi, prsi, rsi3, stop_loss, msr, bcr, d1 = params
    z05_30 = Z05(holding_days=10, rsi=rsi, prsi=prsi, rsi3=rsi3, stop_loss=stop_loss,msr=msr,bcr=bcr,d1=d1)

    summary = await z05_30.backtest("2021-09-01", "2021-09-03")
    summary.update({
        "rsi": rsi,
        "prsi": prsi,
        "rsi3": rsi3,
        "stop_loss": stop_loss,
        "msr": msr,
        "bcr": bcr,
        "d1": d1
    })
    results.append(summary)
df = pd.DataFrame(results)

In [ ]:
columns = ["name", "code", "gains", "buy_at", "sell_at", "duration",  "buy", "sell", "type", "params"]
df = pd.DataFrame(z05_30.orders, columns=columns)

pd.options.display.max_rows = 999
df

In [ ]:
df = pd.DataFrame(results)

In [10]:
results = [(32222, 3180, True, "jqdaptor:18693939333")]
print("   impl   |     account     |   pid  | port | alive")
for pid, port, alive, group in results:
    impl, account = group.split(":")
    alive = "YES" if alive else "*NO*"
    print(f"{impl:10}|{' ':2}{account:15}|  {pid} | {port} | {alive}")

   impl   |     account     |   pid  | port | alive
jqdaptor  |  18693939333    |  32222 | 3180 | YES
